In [ ]:
using System.Net.Http;
using System.Net.NetworkInformation;

In [ ]:
public class BinanceApp
{
    private bool isRunning = true;

    private Task<bool> updateTask = null;

    public void StartApp()
    {
        this.isRunning = true;

        try {
            if (!this.UpdateApp()) {
                Console.WriteLine("Failed to update app.");
            }
            else {
                Console.WriteLine("App updated successfully");
            }
        } catch (Exception e) {
            StringBuilder messageBuilder = new StringBuilder();

            messageBuilder.AppendLine("Failed to update app.");
            messageBuilder.AppendLine($"___MESSAGE:{e.Message}.");
            messageBuilder.AppendLine($"___STACKTRACE:{e.StackTrace}.");
            
            Console.WriteLine(messageBuilder.ToString());
        }

    }

    private bool UpdateApp()
    {
        BinanceService client = new BinanceService();

        Console.WriteLine($"Preparing to update client.");

        this.updateTask = Task.Run<bool>(async () => await client.SendPingAsync());

        Console.WriteLine($"Updated client {this.updateTask.Result.ToString()}.");

        return this.updateTask.Result;
    }
}

public class BinanceService
{
    private const string BASE_URL = "https://api.binance.com";

    public async Task<bool> SendPingAsync()
    {
        Ping ping = new Ping();

        Console.WriteLine($"Preparing to send ping.");

        // Ping url must not contain {https://}.
        PingReply result = await ping.SendPingAsync(BASE_URL.Replace("https://", ""));

        Console.WriteLine($"Ping sent.");

        PingReplyHandler replayHandler = new PingReplyHandler(result);
        replayHandler.LogReply();

        return result.Status == IPStatus.Success;
    }
}

public class PingReplyHandler
{
    public readonly PingReply Reply;

    public PingReplyHandler(PingReply reply)
    {
        this.Reply = reply;
    }

    public void LogReply()
    {
        Console.WriteLine($"Address: {this.Reply.Address}");
        Console.WriteLine($"RoundTrip time: {this.Reply.RoundtripTime}");
        Console.WriteLine($"Buffer: {this.Reply.Buffer.Length}");
        Console.WriteLine($"Status: {this.Reply.Status}");
    }
}

In [ ]:
BinanceApp app = new BinanceApp();
app.StartApp();

Preparing to update client.
Preparing to send ping.
Ping sent.
Address: 13.227.6.143
RoundTrip time: 20
Buffer: 32
Status: Success
Updated client True.
App updated successfully
